<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Problem-Statement" data-toc-modified-id="Problem-Statement-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Problem Statement</a></span></li><li><span><a href="#Data-Description" data-toc-modified-id="Data-Description-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Data Description</a></span></li><li><span><a href="#Proposed-Solution" data-toc-modified-id="Proposed-Solution-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Proposed Solution</a></span><ul class="toc-item"><li><span><a href="#Algorithms" data-toc-modified-id="Algorithms-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Algorithms</a></span></li><li><span><a href="#Evaluating-Model-Performance" data-toc-modified-id="Evaluating-Model-Performance-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Evaluating Model Performance</a></span></li></ul></li></ul></div>

# Problem Statement

We are aiming to put our skills to the test and ...

# Data Description

# Proposed Solution

## Algorithms

## Evaluating Model Performance